In [63]:
from os import listdir
from os.path import isfile, join
import requests
from math import log

In [64]:
DATA_PATH = '../data'
onlyfiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]
url = 'http://localhost:9200/'

In [65]:
#zad1
def get_tagged_words(text_in):
    text= requests.post(url, data = text_in.encode('utf-8'), headers={'Content-type': 'text/plain; charset=utf-8'}).text
    lines = [l for l in text.split('\n') if l != '']
    lines_s= [line.split('\t') for line in lines]
    words = []
    for i in range(1,len(lines_s),2):
        words.append(lines_s[i][1] + ':'+lines_s[i][2].split(':')[0])
    return words 

In [66]:
#zad 2,3
bigrams = {}
iter_count = 0
for filename in onlyfiles:
    file_content = None
    with open(DATA_PATH+'/'+filename,'r',encoding='utf-8') as f:
        file_content=f.read().lower()
    words =  get_tagged_words(file_content)
    for i in range(len(words) - 1):
        bigrams[(words[i],words[i+1])] = bigrams.get((words[i],words[i+1]),0) +1
    iter_count+=1
    if iter_count % 10 ==0:
        print('done {}/{}'.format(iter_count,len(onlyfiles)))


done 10/1180
done 20/1180
done 30/1180
done 40/1180
done 50/1180
done 60/1180
done 70/1180
done 80/1180
done 90/1180
done 100/1180
done 110/1180
done 120/1180
done 130/1180
done 140/1180
done 150/1180
done 160/1180
done 170/1180
done 180/1180
done 190/1180
done 200/1180
done 210/1180
done 220/1180
done 230/1180
done 240/1180
done 250/1180
done 260/1180
done 270/1180
done 280/1180
done 290/1180
done 300/1180
done 310/1180
done 320/1180
done 330/1180
done 340/1180
done 350/1180
done 360/1180
done 370/1180
done 380/1180
done 390/1180
done 400/1180
done 410/1180
done 420/1180
done 430/1180
done 440/1180
done 450/1180
done 460/1180
done 470/1180
done 480/1180
done 490/1180
done 500/1180
done 510/1180
done 520/1180
done 530/1180
done 540/1180
done 550/1180
done 560/1180
done 570/1180
done 580/1180
done 590/1180
done 600/1180
done 610/1180
done 620/1180
done 630/1180
done 640/1180
done 650/1180
done 660/1180
done 670/1180
done 680/1180
done 690/1180
done 700/1180
done 710/1180
done 720/1180
d

In [74]:
import pickle
with open('bigrams_dump' ,'wb') as bd:
    pickle.dump(bigrams, bd )

In [67]:
#zad4
characters = 'aąbcćdeęfghijklłmnńoópqrsśtuvwxyzżź'
bigrams_count_tmp = {}
bigrams_total_value = 0
for key,value in bigrams.items():
    add_to_final = True
    for word in key:
        word = word.split(':')[0]
        if word =='':
            add_to_final = False
            break
        for c in word:
            if not c in characters:
                add_to_final = False
                break
    if add_to_final:
        bigrams_count_tmp[key] = value
        bigrams_total_value += value
bigrams_count_tmp
bigrams= bigrams_count_tmp

In [68]:
bigram_tree_w1 ={}
bigram_tree_w2 ={}
for bigram,count in bigrams.items():
    w1,w2 = bigram
    try:
        bigram_tree_w1[w1][w2] = count
    except KeyError:
        bigram_tree_w1[w1] = {w2:count}
    try:
        bigram_tree_w2[w2][w1] = count
    except KeyError:
        bigram_tree_w2[w2] = {w1:count}


In [69]:
#zad 6b
def H(vals,N):
    out = 0
    for val in vals:
        try:
            val_n = val/N
            if not val_n ==0:
                out+=val_n*log(val_n)
        except ValueError as e:
            print('val : '+str(val))
            print('val_n : '+str(val_n))
            print(vals)
            print('log(val_n) : '+str(log(val_n)))
            raise e
    return out

def LLR_bigram(bigram,bigrams_words_total_occurences):
    w1,w2 = bigram
    k11 =bigram_tree_w1[w1][w2]
    k12 = sum(bigram_tree_w2[w2].values()) - k11
    k21 = sum(bigram_tree_w1[w1].values()) - k11
    k22 = bigrams_words_total_occurences -k11 -k12 - k21
    N =  k11+ k12+ k21+ k22
    return 2 * N * (H([k11,k12,k21,k22],N) - H([k11+k12,k21+k22],N) - H([k11+k21,k12+k22],N))
    
# N=sum(bigram_freq_list_words.values())

llr_bigrams = list(map(lambda x: (x[0],x[1],LLR_bigram(x[0],bigrams_total_value)),bigrams.items()))
llr_bigrams.sort(key= lambda x: -x[2])
llr_bigrams[:10]

[(('który:adj', 'mowa:subst'), 28644, 248954.706021335),
 (('o:prep', 'który:adj'), 28762, 190926.10508849766),
 (('mowa:subst', 'w:prep'), 28579, 177709.21936375732),
 (('w:prep', 'artykuł:brev'), 32184, 114509.06783965782),
 (('otrzymywać:fin', 'brzmienie:subst'), 10586, 111219.30919630887),
 (('w:prep', 'ustęp:brev'), 23564, 88523.27518211817),
 (('minister:subst', 'właściwy:adj'), 7949, 70982.21191642844),
 (('dodawać:fin', 'się:qub'), 8236, 66915.78701422714),
 (('i:conj', 'numer:brev'), 8464, 54505.19718120039),
 (('droga:subst', 'rozporządzenie:subst'), 4754, 54049.0773158387)]

In [70]:
#zad 7,8
partitions = {}
for bigram,occurences in bigrams.items():
    tag0 = bigram[0].split(':')[1]
    tag1 = bigram[1].split(':')[1]
    partitions[(tag0,tag1)] = partitions.get((tag0,tag1),0) + occurences
top_10_partitions =sorted(partitions.items(), key = lambda x: -x[1])[:10]
top_10_partitions

[(('prep', 'subst'), 326059),
 (('subst', 'subst'), 287358),
 (('subst', 'adj'), 274246),
 (('adj', 'subst'), 187820),
 (('subst', 'prep'), 172754),
 (('subst', 'conj'), 84699),
 (('conj', 'subst'), 84220),
 (('prep', 'adj'), 79458),
 (('ger', 'subst'), 77472),
 (('prep', 'brev'), 67230)]

In [71]:
#zad9
top_10_partitions_dicts = { k: [] for k  in list(map(lambda x: x[0],top_10_partitions))}
top_10_partitions_dicts
for bigram,occurance,score in llr_bigrams:
    tag0 = bigram[0].split(':')[1]
    tag1 = bigram[1].split(':')[1]
    for parition in top_10_partitions_dicts.keys():
        if len(top_10_partitions_dicts[parition]) >= 5:
            continue
        if not(tag0 == parition[0] and  tag1  == parition[1]):
            continue
        top_10_partitions_dicts[parition].append((bigram,occurance,score))

top_10_partitions_dicts         
        
            

{('prep', 'subst'): [(('z:prep', 'dzień:subst'), 11416, 53967.24754989901),
  (('na:prep', 'podstawa:subst'), 6727, 47367.51104881132),
  (('do:prep', 'sprawa:subst'), 8734, 46304.54941836767),
  (('w:prep', 'droga:subst'), 7138, 32092.17473576187),
  (('od:prep', 'dzień:subst'), 5359, 31861.06759087903)],
 ('subst',
  'subst'): [(('droga:subst', 'rozporządzenie:subst'),
   4754,
   54049.0773158387), (('skarb:subst', 'państwo:subst'),
   1821,
   21925.49406434114), (('rada:subst', 'minister:subst'),
   2265,
   18309.769926366385), (('terytorium:subst', 'rzeczpospolita:subst'),
   1229,
   14317.429793192), (('ochrona:subst', 'środowisko:subst'),
   1572,
   14028.677468123677)],
 ('subst',
  'adj'): [(('minister:subst', 'właściwy:adj'),
   7949,
   70982.21191642844), (('rzeczpospolita:subst',
    'polski:adj'), 3636, 43656.726644248054), (('jednostka:subst',
    'organizacyjny:adj'),
   2260,
   24600.692689416686), (('samorząd:subst', 'terytorialny:adj'),
   1675,
   23385.2610733

#zad 10

1) What types of bigrams have been found?

('prep', 'subst') ('subst', 'subst')('subst', 'adj')('adj', 'subst')('subst', 'prep')('subst', 'conj')('conj', 'subst')('prep', 'adj')('ger', 'subst')('prep', 'brev'). Just as without tagging, LLR found very commonly occuring bigrams. Even more so than in labs4, cause here we also cut down paritions, to most popular ones, further eliminating important, but not so common (and thus not most obvious),
bigrams. 

2)Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?

here is a lot of nouns in valuable multiword expression (since nouns and adjective tend to bring meaning rather than grammar, and are thus are imporatant). However less usefull, while really popular, are combos of preps with something or abbrevations with something like 'w pkt.' 'w ust.' 'o którym/która etc.'

3) Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?

Since we sorted by LLR score it's hard to say, but it seems like syntactic category would a very good filtration tool. For example it's quite obvious after reading one document that 'w ust.' will be very common, so i don't think a special analisys is necessary to figure that information about given data, and can be easily filtered out in order to look for less obious pairs.

4) Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem


We could probably to try it to create better translators. Some words would imply diffrenet meanings. 





